In [ ]:
#Imports same as final_store.py
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import streamlit as st
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from PIL import Image
load_dotenv()

In [ ]:
# Call contract address
contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

In [ ]:
print(contract_address)

In [ ]:
#Create w3 object
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

In [ ]:
# Load Contract
def load_contract(w3):

    # Load the contract ABI
    with open(Path('./contracts/compiled/mem_token_abi.json')) as f:
        mem_token_abi = json.load(f)

    contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

    # Load the contract
    contract = w3.eth.contract(
        address=contract_address,
        abi=mem_token_abi
    )

    return contract

contract = load_contract(w3)

In [ ]:
print(contract)

In [ ]:
# Call all ganache accounts
all_accounts = w3.eth.accounts
all_accounts

In [ ]:
#Import faker to generate fake customer data
from faker import Faker
fake = Faker()



# Connect to database
connection = psycopg2.connect(
    dbname="fintech_project_3",
    user="postgres",
    password="postgres",
    host="localhost",
    port=5432
)

cursor = connection.cursor()

#Manually set traitindex for each round of demo purchases
traitIndex = 4

# For loop to generate customer information, mintMembership, and insert to database
# Enter discretionary number for range value
for i in range(47):
    # Generate fake customer information
    first_name = fake.first_name()
    last_name = fake.last_name()
    street_address = fake.street_address()
    city = fake.city()
    state = fake.state()
    zip_code = fake.postalcode()
    
    # Choose account with balance less than 150 ETH (dont want the contract owner account)
    # and greater than 1 ETH (so they can afford the purchase)
    for i in all_accounts:
        #Get account balance
        wall_bal = float(round(w3.fromWei(w3.eth.get_balance(i), "ether"),2))
        if wall_bal < 150:
            if wall_bal > 1:
                address = i
    
    # Call trait price in wei for selected traitIndex
    trait_price_wei = w3.toWei(contract.functions.getMembershipInfo(traitIndex).call()[1], 'wei')
    # Convert price to ETH
    trait_price_eth= w3.fromWei(trait_price_wei, 'ether') 
    
    # Use the contract to send a transaction to the mintMembership function
    tx_hash = contract.functions.mintMembership(
        address,
        traitIndex
    ).transact({'from': address, 'gas': 1000000, 'value': trait_price_wei})
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)

    # Store other values for db insert
    tx_hash_value=receipt['transactionHash'].hex()
    traitIndex_value=traitIndex
    eth_price_value=trait_price_eth
    gas_cost_value= receipt['gasUsed']

    insert_query = """
    INSERT INTO customer_data (first_name, last_name, street_address, city, cust_state, zip_code, tx_hash, traitIndex, eth_price, gas_cost)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    data_to_insert = (first_name, last_name, street_address, city, state, zip_code, tx_hash_value, traitIndex_value, eth_price_value, gas_cost_value)

    cursor.execute(insert_query, data_to_insert)
    connection.commit()
    

cursor.close()
connection.close()